In [2]:
# from datasets import load_dataset, load_from_disk
# hf_dataset = load_dataset("nielsr/CelebA-faces", split="train")
# hf_dataset.save_to_disk("./celeba_local")

/home/latim/PycharmProjects/WatDNN/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Saving the dataset (3/3 shards): 100%|██████████| 202599/202599 [00:01<00:00, 138266.47 examples/s]


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import psutil


class VAE(nn.Module):
    def __init__(self, latent_dim=200):
        super(VAE, self).__init__()

        # Encoder: 4 Conv2d layers with stride=2, kernel=4, padding=1
        # Input: (3, 64, 64) -> (32, 32, 32) -> (64, 16, 16) -> (128, 8, 8) -> (256, 4, 4)
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=4, stride=2, padding=1),    # 0
            nn.ReLU(),                                                # 1
            nn.Conv2d(32, 64, kernel_size=4, stride=2, padding=1),   # 2
            nn.ReLU(),                                                # 3
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1),  # 4
            nn.ReLU(),                                                # 5
            nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1), # 6
            nn.ReLU(),                                                # 7
        )

        # 256 * 4 * 4 = 4096
        self.fc_mu = nn.Linear(4096, latent_dim)
        self.fc_logvar = nn.Linear(4096, latent_dim)

        # Decoder input
        self.decoder_input = nn.Linear(latent_dim, 4096)

        # Decoder: 4 ConvTranspose2d layers
        # (256, 4, 4) -> (128, 8, 8) -> (64, 16, 16) -> (32, 32, 32) -> (3, 64, 64)
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1),  # 0
            nn.ReLU(),                                                          # 1
            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1),   # 2
            nn.ReLU(),                                                          # 3
            nn.ConvTranspose2d(64, 32, kernel_size=4, stride=2, padding=1),    # 4
            nn.ReLU(),                                                          # 5
            nn.ConvTranspose2d(32, 3, kernel_size=4, stride=2, padding=1),     # 6
            nn.Sigmoid(),                                                       # 7
        )

    def encode(self, x):
        x = self.encoder(x)
        x = x.view(x.size(0), -1)  # Flatten to (batch, 4096)
        mu = self.fc_mu(x)
        logvar = self.fc_logvar(x)
        return mu, logvar

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z):
        x = self.decoder_input(z)
        x = x.view(x.size(0), 256, 4, 4)  # Reshape to (batch, 256, 4, 4)
        x = self.decoder(x)
        return x

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        recon = self.decode(z)
        return recon, mu, logvar


In [2]:
import torch
import torch.nn as nn
from diffusers import DDPMPipeline, DDPMScheduler, UNet2DModel
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from tqdm import tqdm
from copy import deepcopy
import torch.nn.functional as F
from torch.optim import AdamW
from datasets import load_dataset

# --- Classe Principale Uchida ---

class UchidaVAE:
    def __init__(self, model, device="cuda"):
        self.device = device
        self.model= model

        # Configuration par défaut
        self.config = {
            "layer_name": "decoder.6.weight", # Cible robuste
            "watermark_len":16,
            "lr": 1e-4,
            "lambda_wat": 1.0,
            "epochs":50,
            "beta_kl":1.0
        }

        self.saved_keys = {}

    def _get_target_weights(self, model):
        """Récupère le tenseur des poids de la couche cible."""
        for name, param in model.named_parameters():
            if name == self.config["layer_name"]:
                return param
        raise ValueError(f"Paramètre {self.config['layer_name']} introuvable.")

    def embed(self, dataloader):
        """
        Incorpore la marque Uchida (Régularisation des poids) pendant le finetuning.
        """
        print(f"--- start Embedding UCHIDA in VAE({self.config['layer_name']}) ---")

        wm_model = self.model
        wm_model.train()
        

        # 1. Génération des Clés (Matrice A et Watermark binaire)
        # On récupère la dimension des poids pour initialiser A
        target_weights = self._get_target_weights(wm_model)
        print(target_weights.shape)

        # Uchida : Moyenne sur l'axe des canaux de sortie (ou aplatissement total)
        # Ici on suit la logique standard : mean(0) puis flatten
        with torch.no_grad():
            w_flat_dim = torch.flatten(target_weights.mean(dim=0)).shape[0]

        print(f"Dimension vecteur poids : {w_flat_dim} | Watermark : {self.config['watermark_len']} bits")

        matrix_a = torch.randn(w_flat_dim, self.config["watermark_len"]).to(self.device)
        watermark_target = torch.randint(0, 2, (self.config["watermark_len"],)).float().to(self.device)


        # 2. Optimiseur
        optimizer = torch.optim.AdamW(wm_model.parameters(), lr=self.config["lr"])
        mse_loss = nn.MSELoss()
        bce_loss = nn.BCELoss()

        # 3. Boucle d'entraînement
        for epoch in range(self.config["epochs"]):
            pbar = tqdm(dataloader)
            for clean_images, _ in pbar:
                clean_images = clean_images.to(self.device)

                optimizer.zero_grad()

                recon, mu, logvar = wm_model(clean_images)

                # Reconstruction loss
                l_recon = F.mse_loss(recon, clean_images)

                # KL divergence
                l_kl = -0.5 * torch.mean(1 + logvar - mu.pow(2) - logvar.exp())

                # VAE total task loss
                l_main = l_recon + self.config["beta_kl"] * l_kl

                # B. Watermark Loss (Regularization)
                current_weights = self._get_target_weights(wm_model)  # e.g. model.decoder[6].weight
                w_flat = torch.flatten(current_weights.mean(dim=0))




                pred_wm_prob = torch.sigmoid(w_flat @ matrix_a)
                l_wat = bce_loss(pred_wm_prob, watermark_target)

                # Loss Totale
                l_total = l_main + self.config["lambda_wat"] * l_wat

                l_total.backward()
                optimizer.step()

                # Metrics
                ber = self._compute_ber(pred_wm_prob, watermark_target)
                ram_gb = psutil.Process().memory_info().rss / 1e9
                pbar.set_description(f"epoch:{epoch}  |  Recon: {l_recon:.3f} | KL: {l_kl:.3f} | L_Wat: {l_wat:.3f} | BER: {ber:.2f}  RAM:{ram_gb:.1f}GB")

                # if ber == 0.0 and l_wat.item() < 0.01:
                #     print("✅ Convergence atteinte !")
                #     break
            if ber == 0.0 and epoch>=9: break

        # Sauvegarde des clés
        self.saved_keys = {
            "matrix_a": matrix_a,
            "watermark_target": watermark_target,
            "watermarked_model": wm_model
        }
        torch.save(self.saved_keys, "Uchida_VAE_model_checkpoint.pt")
        return wm_model

    def extract(self, model=None):
        """
        Extrait la marque d'un modèle suspect (lecture des poids).
        """
        if model is None:
            model = self.saved_keys["watermarked_model"]

        matrix_a = self.saved_keys["matrix_a"]
        watermark_target = self.saved_keys["watermark_target"]

        # 1. Récupération des poids
        try:
            target_weights = self._get_target_weights(model)
        except ValueError:
            print("⚠️ Couche cible introuvable dans le modèle suspect.")
            return 1.0 # BER max

        # 2. Projection
        with torch.no_grad():
            w_flat = torch.flatten(target_weights.mean(dim=0))
            pred_wm_prob = torch.sigmoid(w_flat @ matrix_a)

            ber = self._compute_ber(pred_wm_prob, watermark_target)

        print(f"BER Extrait : {ber:.2f}")
        return ber, pred_wm_prob

    @staticmethod
    def _compute_ber(pred, target):
        return ((pred > 0.5).float() != target).float().mean().item()




/home/latim/PycharmProjects/WatDNN/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
from datasets import load_dataset
from torch.utils.data import DataLoader
import torch
from torchvision import transforms

import os
import torch
import gc
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Load from Hugging Face (no Google Drive issues)
print("Loading dataset...")


from datasets import load_from_disk
hf_dataset = load_from_disk("celeba_local")
# os.makedirs("./celeba_images/all", exist_ok=True)
# for i, item in enumerate(hf_dataset):
#     item['image'].save(f"./celeba_images/all/{i:06d}.jpg")
# del hf_dataset


class CelebAWrapper(torch.utils.data.Dataset):
    def __init__(self, hf_dataset, transform):
        self.dataset = hf_dataset
        self.transform = transform

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        image = self.dataset[idx]['image']
        if self.transform:
            image = self.transform(image)
        return image, 0

dataset = CelebAWrapper(hf_dataset, transform)
# del hf_dataset
# gc.collect()
# dataset = datasets.ImageFolder("./celeba_images", transform=transform)
print("Dataset loaded!")

dataloader = DataLoader(dataset, batch_size=64, shuffle=True)
print("loader loaded!")


/home/latim/PycharmProjects/WatDNN/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading dataset...
Dataset loaded!
loader loaded!


In [4]:

#load model
latent_dim = 200

# Initialize the model
model = VAE(latent_dim=latent_dim)

# Load the trained weights
model_path = "./vae_celeba_latent_200_epochs_10_batch_64_subset_80000.pth"
model.load_state_dict(torch.load(model_path))
model.to(device)




uchida_VAE = UchidaVAE(model)
print("Uchida VAE loaded!")
watermarked_model = uchida_VAE.embed(dataloader)

hello
Uchida VAE loaded!
--- start Embedding UCHIDA in VAE(decoder.6.weight) ---
torch.Size([32, 3, 4, 4])
Dimension vecteur poids : 48 | Watermark : 16 bits


epoch:0  |  Recon: 0.284 | KL: 0.019 | L_Wat: 0.116 | BER: 0.00  RAM:2.1GB: 100%|██████████| 3166/3166 [00:31<00:00, 101.45it/s]
epoch:1  |  Recon: 0.291 | KL: 0.020 | L_Wat: 0.029 | BER: 0.00  RAM:2.1GB: 100%|██████████| 3166/3166 [00:40<00:00, 78.02it/s]
epoch:2  |  Recon: 0.251 | KL: 0.021 | L_Wat: 0.008 | BER: 0.00  RAM:2.1GB: 100%|██████████| 3166/3166 [00:42<00:00, 74.64it/s] 
epoch:3  |  Recon: 0.273 | KL: 0.017 | L_Wat: 0.002 | BER: 0.00  RAM:2.1GB: 100%|██████████| 3166/3166 [00:44<00:00, 71.01it/s]
epoch:4  |  Recon: 0.321 | KL: 0.015 | L_Wat: 0.001 | BER: 0.00  RAM:2.1GB: 100%|██████████| 3166/3166 [00:43<00:00, 72.93it/s]
epoch:5  |  Recon: 0.285 | KL: 0.015 | L_Wat: 0.000 | BER: 0.00  RAM:2.1GB: 100%|██████████| 3166/3166 [00:43<00:00, 73.54it/s]
epoch:6  |  Recon: 0.323 | KL: 0.022 | L_Wat: 0.000 | BER: 0.00  RAM:2.1GB: 100%|██████████| 3166/3166 [00:39<00:00, 79.21it/s]
epoch:7  |  Recon: 0.310 | KL: 0.019 | L_Wat: 0.000 | BER: 0.00  RAM:2.1GB: 100%|██████████| 3166/3166

In [ ]:
# Print current GPU usage at any point

import psutil
print(f"GPU allocated: {torch.cuda.memory_allocated() / 1e9:.2f} GB")
print(f"GPU reserved:  {torch.cuda.memory_reserved() / 1e9:.2f} GB")
ram_gb = psutil.Process().memory_info().rss / 1e9
print(f"RAM used: {ram_gb:.2f} GB")
gc.collect()
torch.cuda.empty_cache()
total_ram = psutil.virtual_memory().total / 1e9
used_ram = psutil.Process().memory_info().rss / 1e9
print(f"RAM: {used_ram:.1f} / {total_ram:.1f} GB")

In [14]:
# --- Fonction de Distillation (Attaque) ---
import gc
import torch.optim as optim

def run_distillation_attack_uchida(uchida_obj, dataloader, epochs=5, lr=1e-3):
    """
    Tente de transférer la fonctionnalité du modèle Uchida vers un modèle vierge.
    Vérifie si la marque (basée sur les poids) survit.
    """
    print(lr)
    device = uchida_obj.device
    checkpoint = torch.load("Uchida_VAE_model_checkpoint.pt", weights_only=False)

    # 1. Teacher (Gelé)
    teacher = checkpoint["watermarked_model"]
    teacher.eval()
    for p in teacher.parameters(): p.requires_grad = False

    # 2. Student (Vierge - Même architecture)
    print("\n--- Initialisation du Student ---")
    # student_unet = UNet2DModel.from_config(teacher_unet.config).to(device)
    # student_pipeline = DDPMPipeline.from_pretrained("google/ddpm-cifar10-32")
    student=VAE(latent_dim=latent_dim)
    student.load_state_dict(torch.load(model_path))
    student.to(device)
    student.train()

    teacher_ber, _ = uchida_obj.extract(teacher)
    student_ber, _ = uchida_obj.extract(student)
    # Sanity Checks
    print(f"[Check] BER Teacher: {teacher_ber:.2f}")
    print(f"[Check] BER Student (Avant): {student_ber:.2f}")

    optimizer = AdamW(student.parameters(), lr=lr)
    scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[1], gamma=0.1)

    # history = {"loss": [], "ber": []}

    print(f"\n--- Distillation Uchida ({epochs} epochs) ---")
    a=0
    history=0

    for epoch in range(epochs):
        pbar = tqdm(dataloader, desc=f"Epoch {epoch+1}")
        running_loss = 0.0

        for clean_images, _ in pbar:
            clean_images = clean_images.to(device)

          #B. Distillation (Output Matching)
            with torch.no_grad():
                target_pred,_,_ = teacher(clean_images)

            student_pred,_,_ = student(clean_images)

            loss = F.mse_loss(student_pred, target_pred)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            pbar.set_postfix(Loss=loss.item())

        # C. Vérification : Est-ce que les poids se sont alignés sur Uchida ?
        current_ber, pred_wm_prob = uchida_obj.extract(student)
        scheduler.step()
        # history["ber"].append(current_ber)
        

        # Monitor memory each step

        ram_gb = psutil.Process().memory_info().rss / 1e9
        print(f"👉 Fin Epoch {epoch+1} | Loss: {loss.item():.4f} | BER Student: {current_ber:.2f} | err_wat: {nn.BCELoss()(pred_wm_prob, checkpoint["watermark_target"]).item()}  GPU:{torch.cuda.memory_allocated() / 1e9:.1f}GB RAM:{ram_gb:.1f}GB")
        if current_ber==0.0 and a>=1:
            print("✅ Marque récupérée avec succès par distillation !")
            break
        elif current_ber==0.0 and a<1 :
            a+=1
        else:
            a=0
        gc.collect()
        torch.cuda.empty_cache()

    return student, history



In [ ]:

student_res, stats = run_distillation_attack_uchida(uchida_VAE, dataloader, epochs=1000)



0.001

--- Initialisation du Student ---
BER Extrait : 0.00
BER Extrait : 0.38
[Check] BER Teacher: 0.00
[Check] BER Student (Avant): 0.38

--- Distillation Uchida (1000 epochs) ---


Epoch 1: 100%|██████████| 3166/3166 [00:27<00:00, 114.20it/s, Loss=0.00779]


BER Extrait : 0.12
👉 Fin Epoch 1 | Loss: 0.0078 | BER Student: 0.12 | err_wat: 0.6727325320243835  GPU:0.3GB RAM:2.2GB


Epoch 2: 100%|██████████| 3166/3166 [00:25<00:00, 124.42it/s, Loss=0.00734]


BER Extrait : 0.12
👉 Fin Epoch 2 | Loss: 0.0073 | BER Student: 0.12 | err_wat: 0.6737686991691589  GPU:0.3GB RAM:2.2GB


...

Epoch 105: 100%|██████████| 3166/3166 [00:39<00:00, 80.03it/s, Loss=0.0071] 


BER Extrait : 0.06
👉 Fin Epoch 105 | Loss: 0.0071 | BER Student: 0.06 | err_wat: 0.6625863909721375  GPU:0.3GB RAM:2.3GB


Epoch 106: 100%|██████████| 3166/3166 [00:38<00:00, 81.36it/s, Loss=0.00645]


BER Extrait : 0.06
👉 Fin Epoch 106 | Loss: 0.0064 | BER Student: 0.06 | err_wat: 0.6624147891998291  GPU:0.3GB RAM:2.3GB


Epoch 107:  40%|███▉      | 1265/3166 [00:15<00:23, 80.21it/s, Loss=0.00606]